# Part-of-Speech Tagging for Danish

<div class="admonition note" name="html-admonition" style="background: lightblue; padding: 10px">
<p class="title">Note</p>
This section, "Working in Languages Beyond English," is co-authored with <a href="http://www.quinndombrowski.com/">Quinn Dombrowski</a>, the Academic Technology Specialist at Stanford University and a leading voice in multilingual digital humanities. I'm grateful to Quinn for helping expand this textbook to serve languages beyond English. 
</div>

In this lesson, we're going to learn about the textual analysis methods *part-of-speech tagging* and *keyword extraction* for Danish texts. These methods will help us computationally parse sentences and better understand words in context.

---

## spaCy and Natural Language Processing (NLP)

To computationally identify parts of speech, we're going to use the natural language processing library spaCy. For a more extensive introduction to NLP and spaCy, see the previous lesson.

To parse sentences, spaCy relies on machine learning models that were trained on large amounts of labeled text data. If you've used the preprocessing or named entity recognition notebooks for this language, you can skip the steps for installing spaCy and downloading the language model.

## Install spaCy

To use spaCy, we first need to install the library.

In [ ]:
!pip install -U spacy

## Import Libraries

Then we're going to import `spacy` and `displacy`, a special spaCy module for visualization.

In [2]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 400
pd.options.display.max_colwidth = 400

We're also going to import the `Counter` module for counting nouns, verbs, adjectives, etc., and the `pandas` library for organizing and displaying data (we're also changing the pandas default max row and column width display setting).

## Download Language Model

Next we need to download the Danish-language model (`da_core_news_md`), which will be processing and making predictions about our texts. This is the model that was trained on a corpus of news and nonfiction. You can download the `da_core_news_md` model by running the cell below:

In [3]:
!python -m spacy download da_core_news_md

     ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
     ---------------------------------------- 0.1/42.2 MB 3.3 MB/s eta 0:00:13
     ---------------------------------------- 0.3/42.2 MB 4.1 MB/s eta 0:00:11
      --------------------------------------- 0.7/42.2 MB 5.5 MB/s eta 0:00:08
     - -------------------------------------- 1.5/42.2 MB 8.4 MB/s eta 0:00:05
     -- ------------------------------------- 2.5/42.2 MB 11.5 MB/s eta 0:00:04
     --- ------------------------------------ 3.7/42.2 MB 13.9 MB/s eta 0:00:03
     ---- ----------------------------------- 4.8/42.2 MB 15.5 MB/s eta 0:00:03
     ----- ---------------------------------- 6.0/42.2 MB 16.7 MB/s eta 0:00:03
     ------ --------------------------------- 7.2/42.2 MB 17.8 MB/s eta 0:00:02
     ------- -------------------------------- 8.4/42.2 MB 18.5 MB/s eta 0:00:02
     -------- ------------------------------- 9.5/42.2 MB 18.9 MB/s eta 0:00:02
     ---------- ---------------------------- 10.9/42

*Note: spaCy offers [models for other languages](https://spacy.io/usage/models#languages) including Chinese, German, French, Spanish, Portuguese, Russian, Italian, Dutch, Danish, Greek, Norwegian, and Lithuanian*.  

*spaCy offers language and tokenization support for other language via external dependencies — such as [PyviKonlpy](https://github.com/konlpy/konlpy) for Korean.*

## Load Language Model

Once the model is downloaded, we need to load it with `spacy.load()` and assign it to the variable `nlp`.

In [4]:
nlp = spacy.load('da_core_news_md')

## Create a Processed spaCy Document

Whenever we use spaCy, our first step will be to create a processed spaCy `document` with the loaded NLP model `nlp()`. Most of the heavy NLP lifting is done in this line of code. After processing, the `document` object will contain tons of juicy language data — named entities, sentence boundaries, parts of speech — and the rest of our work will be devoted to accessing this information.

In [5]:
filepath = '../texts/da.txt'
text = open(filepath, encoding='utf-8').read()
document = nlp(text)

## spaCy Part-of-Speech Tagging
The tags that spaCy uses for part-of-speech are based on work done by [Universal Dependencies](https://universaldependencies.org/), an effort to create a set of part-of-speech tags that work across many different languages. Texts from various languages are annotated using this common set of tags, and contributed to a common repository that can be used to train models like spaCy.

The Universal Dependencies page has information about the annotated corpora available for each language; it's worth looking into the corpora that were annotated for your language.

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |
| ADP   | adposition                | in, to, during                                |
| ADV   | adverb                    | very, tomorrow, down, where, there            |
| AUX   | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ  | conjunction               | and, or, but                                  |
| CCONJ | coordinating conjunction  | and, or, but                                  |
| DET   | determiner                | a, an, the                                    |
| INTJ  | interjection              | psst, ouch, bravo, hello                      |
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |
| NUM   | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART  | particle                  | ’s, not,                                      |
| PRON  | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT | punctuation               | ., (, ), ?                                    |
| SCONJ | subordinating conjunction | if, while, that                               |
| SYM   | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :), 😝             |
| VERB  | verb                      | run, runs, running, eat, ate, eating          |
| X     | other                     | sfpksdpsxmsa                                  |
| SPACE | space                     |                                               |


Above is a POS chart taken from [spaCy's website](https://spacy.io/api/annotation#named-entities), which shows the different parts of speech that spaCy can identify as well as their corresponding labels. To quickly see spaCy's POS tagging in action, we can use the [spaCy module `displacy`](https://spacy.io/usage/visualizers#ent) on our sample `document` with the `style=` parameter set to "dep" (short for dependency parsing):

## Get Part-Of-Speech Tags

To get part of speech tags for every word in a document, we have to iterate through all the tokens in the document and pull out the `.lemma_` attribute for each token, which gives us the un-inflected version of the word. We'll also pull out the  `.pos_` attribute for each token. We can get even finer-grained dependency information with the attribute `.dep_`.


In [6]:
for token in document:
    print(token.lemma_, token.pos_, token.dep_)

EVANGELINES NOUN ROOT

  SPACE dep
GENVORDIGHEDER NOUN nmod


  SPACE dep
tIL ADP nmod

  SPACE dep
KVINDER NOUN ROOT
med ADP case
RØDT NOUN nmod
HAAR ADJ advmod





  SPACE dep
ELINOR PROPN conj
GLYN PROPN flat


  SPACE dep
EVANGELINES X appos

  SPACE dep
GENVORDIGHEDER NOUN appos


  SPACE dep
AUTORISERET PROPN appos
OVERSÆTTELSE NOUN appos

  SPACE dep
FOR ADP case
NORGE PROPN nmod
og CCONJ cc
DANMARK PROPN conj
AF ADP case

  SPACE dep
HEDVIG PROPN ROOT
MAGNUSSEN NOUN flat


  SPACE dep
NY NOUN punct
UDGAVE ADJ ROOT


  SPACE dep
MARTINS PROPN advmod
FORLAG NOUN nsubj

  SPACE dep
KØBENHAVN PROPN appos
& CCONJ cc
KRISTIANIA PROPN conj

  SPACE dep
MCMXXI PROPN ROOT





  SPACE dep
MARTIN'S PROPN nmod:poss
FORLAGSTRYKKERI PROPN nmod:poss
, PUNCT punct
KØBENHAVN PROPN conj





 SPACE dep
BEGYNDELSEN X flat
pAA NOUN nmod:poss
EVANGELINES PROPN amod
DAGBOG X ROOT



                                                SPACE dep
_ NOUN nmod
branches NOUN ROOT
Park PROPN obj
_ NOUN flat


## Practicing with the example text
When working with languages that have inflection, we typically use `token.lemma_` instead of `token.text` like you'll find in the English examples. This is important when we're counting, so that differently-inflected forms of a word (e.g. masculine vs. feminine or singular vs. plural) aren't counted as if they were different words.

In [7]:
filepath = "../texts/da.txt"
document = nlp(open(filepath, encoding="utf-8").read())

## Get Adjectives

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |

To extract and count the adjectives in the example text, we will follow the same model as above, except we'll add an `if` statement that will pull out words only if their POS label matches "ADJ."

<div class="admonition pythonreview" name="html-admonition" style="background: lightgreen; padding: 10px">
<p class="title">Python Review</p>

While we demonstrate how to extract parts of speech in the sections below, we're also going to reinforce some integral Python skills. Notice how we use `for` loops and `if` statements to `.append()` specific words to a list. Then we count the words in the list and make a pandas dataframe from the list.
</div>

Here we make a list of the adjectives identified in the example text:

In [8]:
adjs = []
for token in document:
    if token.pos_ == 'ADJ':
        adjs.append(token.lemma_)

In [9]:
adjs

['HAAR',
 'UDGAVE',
 '3.',
 'vild',
 'morsom',
 'Eventyrerske',
 'øjensynlig',
 'al',
 'mulig',
 'inkonsekvent',
 'ordentlig',
 'al',
 'forrige',
 'syg',
 'samme',
 'behagelig',
 'god',
 'død',
 'virkelig',
 'meget',
 'umulig',
 'egen',
 'meget',
 'død',
 'gift',
 'død',
 'meget',
 'al',
 'formell',
 'venskabelig',
 'meget',
 'mange',
 'gammel',
 'ung',
 'mørk',
 'vild',
 'vild',
 'maask',
 'smuk',
 'rar',
 'vild',
 'passende',
 'sidste',
 'forhade',
 'gammel',
 'nødt',
 'ondt',
 'tiltalende',
 'mulig',
 'Eventyrerske',
 'god',
 'Fine',
 'vild',
 'al',
 'smuk',
 'ophøjet',
 'ædel',
 'stor',
 'behagelig',
 'værd',
 'heldig',
 'latterlig',
 'hvid',
 'frygtelig',
 'ondskabsfuld',
 'al',
 'lille',
 'døv',
 'døve',
 'rasende',
 'rask',
 'vild',
 'næste',
 'sidste',
 'død',
 'Vret',
 'gammel',
 'Eventyrerske',
 'rig',
 'unyttig',
 'al',
 'tilstrækkeligt',
 'grim',
 'kedelig',
 'gift',
 'meget',
 'gammel',
 'ækl',
 'mange',
 'lille',
 'underligst',
 'vis',
 'sidste',
 'ung',
 'menneskelig',
 

Then we count the unique adjectives in this list with the `Counter()` module:

In [10]:
adjs_tally = Counter(adjs)

In [11]:
adjs_tally.most_common()

[('vild', 272),
 ('al', 208),
 ('god', 107),
 ('meget', 102),
 ('gammel', 67),
 ('hel', 64),
 ('stor', 61),
 ('lille', 60),
 ('smuk', 52),
 ('mange', 42),
 ('gift', 40),
 ('ung', 40),
 ('kær', 33),
 ('sød', 32),
 ('egen', 31),
 ('glad', 31),
 ('samme', 29),
 ('mulig', 26),
 ('sikker', 26),
 ('rød', 25),
 ('kold', 25),
 ('maask', 24),
 ('kedelig', 24),
 ('første', 24),
 ('sidste', 23),
 ('dejlig', 22),
 ('høj', 21),
 ('sandt', 21),
 ('bange', 21),
 ('eneste', 20),
 ('lykkelig', 20),
 ('vred', 20),
 ('_', 20),
 ('yndig', 19),
 ('ulykkelig', 19),
 ('umulig', 18),
 ('nødt', 18),
 ('maaske', 17),
 ('elskværdig', 17),
 ('underlig', 16),
 ('venlig', 16),
 ('behagelig', 15),
 ('sort', 15),
 ('lang', 15),
 ('færdig', 15),
 ('dum', 15),
 ('slem', 14),
 ('ny', 13),
 ('lidt', 12),
 ('nervøs', 12),
 ('forskellig', 12),
 ('anderledes', 12),
 ('rædsom', 11),
 ('gal', 11),
 ('vidunderlig', 11),
 ('morsom', 10),
 ('Eventyrerske', 10),
 ('vis', 10),
 ('halv', 10),
 ('hvid', 9),
 ('frygtelig', 9),
 ('rig

Then we make a dataframe from this list:

In [12]:
df = pd.DataFrame(adjs_tally.most_common(), columns=['adj', 'count'])
df[:100]

,adj,count
0,vild,272
1,al,208
2,god,107
3,meget,102
4,gammel,67
5,hel,64
6,stor,61
7,lille,60
8,smuk,52
9,mange,42


## Get Nouns

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |

To extract and count nouns, we can follow the same model as above, except we will change our `if` statement to check for POS labels that match "NOUN".

In [13]:
nouns = []
for token in document:
    if token.pos_ == 'NOUN':
        nouns.append(token.lemma_)

nouns_tally = Counter(nouns)

df = pd.DataFrame(nouns_tally.most_common(), columns=['noun', 'count'])
df[:100]

,noun,count
0,kunde,190
1,--,109
2,_,103
3,Aa,78
4,tid,56
5,Aften,55
6,Øjne,50
7,Mand,49
8,ansigt,45
9,Gang,43


## Get Verbs

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| VERB  | verb                      | run, runs, running, eat, ate, eating          |

To extract and count works of art, we can follow a similar-ish model to the examples above. This time, however, we're going to make our code even more economical and efficient (while still changing our `if` statement to match the POS label "VERB").

<div class="admonition pythonreview" name="html-admonition" style="background: lightgreen; padding: 10px">
<p class="title">Python Review</p>

We can use a [*list comprehension*](https://melaniewalsh.github.io/Intro-Cultural-Analytics/Python/More-Lists-Loops.html#List-Comprehensions) to get our list of verbs in a single line of code! Closely examine the first line of code below:
</div>

In [14]:
verbs = [token.lemma_ for token in document if token.pos_ == 'VERB']

verbs_tally = Counter(verbs)

df = pd.DataFrame(verbs_tally.most_common(), columns=['verb', 'count'])
df[:100]

,verb,count
0,sige,741
1,have,374
2,komme,322
3,være,287
4,se,256
5,gøre,234
6,tage,132
7,véd,118
8,faa,110
9,tale,103


# Keyword Extraction

## Get Sentences with Keyword

spaCy can also identify sentences in a document. To access sentences, we can iterate through `document.sents` and pull out the `.text` of each sentence.

We can use spaCy's sentence-parsing capabilities to extract sentences that contain particular keywords, such as in the function below. Note that the function assumes that the keyword provided will be exactly the same as it appears in the text (e.g. matching all inflection for case, number, gender, etc. As a Spanish example, if you use `bueno` as the keyboard, it won't match `buena` or `buenos`.)

With the function `find_sentences_with_keyword()`, we will iterate through `document.sents` and pull out any sentence that contains a particular "keyword." Then we will display these sentence with the keywords bolded.

In [15]:
import re
from IPython.display import Markdown, display

In [16]:
def find_sentences_with_keyword(keyword, document):
    
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        sentence = sentence.text
        
        #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
        if keyword.lower() in sentence.lower():
            
            #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            sentence = re.sub('\n', ' ', sentence)
            sentence = re.sub(f"{keyword}", f"**{keyword}**", sentence, flags=re.IGNORECASE)
            
            display(Markdown(sentence))

In [17]:
find_sentences_with_keyword(keyword="godt", document=document)

Jeg har læst alt muligt om det i en Bog; det er at se **godt** ud, og ikke at have noget at leve af, og dog have Fornøjelse af Livet -- og det har jeg i Sinde!

Jeg har ganske vist ingenting at leve af, for man kan ikke regne 300 Pund om Aaret for noget videre -- jeg er overordentlig køn, og jeg véd det **godt**, og jeg forstaar at sætte mit Haar og tage mine Hatte paa og den Slags Ting, saa jeg er naturligvis Eventyrerske!

Jeg er tyve Aar, og lige indtil forrige Uge, da Fru Carruthers blev syg og døde paa én og samme Dag, havde jeg det sommetider meget behageligt, naar hun var i **godt** Humør.  

Naar det bliver mørkt, og jeg er alene her oppe, spekulerer jeg ofte paa, hvorledes det vilde have været, hvis jeg havde haft nogle -- men jeg tror, at jeg er af den Slags Katte, som ikke vilde have kommet videre **godt** ud af det med dem -- saa det er maaske bedst saaledes, hvis jeg bare havde -- for Eksempel en smuk Tante til at holde af mig, det kunde have været rart.  

Jeg er ikke af en Type, som enhver synes **godt** om.

...

"Kort og **godt**, han har absolut nægtet at have noget at gøre med Sagen; han siger, at jeg skal ikke vente noget mere fra ham, og vi er skilt fra hinanden for bestandig!"  "Aa, Robert!" udbrød Lady Merrenden forfærdet.  Robert lagde sine Arme omkring mig, og hans Ansigtsudtryk forandredes til straalende Glæde.  

"  "Jeg forstaar mig forholdsvis **godt** paa at bedømme Karakterer.

Jeg vil give ham mig selv, og vi skal komme meget **godt** ud af det uden Dem!"

Saa prøvede jeg paa at fortælle det, saa **godt** jeg kunde, og de hørte aandeløst efter.

Det er altsammen vidunderligt.  Hvad i Alverden er noget som helst andet **godt** for i Livet, end at være vanvittig forelsket, saaledes som vi er det.  

## Get Keyword in Context

We can also find out about a keyword's more immediate context — its neighboring words to the left and right — and we can fine-tune our search with POS tagging.

To do so, we will first create a list of what's called *ngrams*. "Ngrams" are any sequence of *n* tokens in a text. They're an important concept in computational linguistics and NLP. (Have you ever played with [Google's *Ngram* Viewer](https://books.google.com/ngrams)?)

Below we're going to make a list of *bigrams*, that is, all the two-word combinations from the sample text. We're going to use these bigrams to find the neighboring words that appear alongside particular keywords.

In [18]:
#Make a list of tokens and POS labels from document if the token is a word 
tokens_and_labels = [(token.text, token.pos_) for token in document if token.is_alpha]

In [19]:
#Make a function to get all two-word combinations
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    #Loop through numbers from 0 to the (slightly adjusted) length of your word list
    for word_index in range(adj_length_of_word_list):
        
        #Index the list at each number, grabbing the word at that number index as well as N number of words after it
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        #Append this word combo to the master list "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [20]:
bigrams = get_bigrams(tokens_and_labels)

Let's take a peek at the bigrams:

In [21]:
bigrams[5:20]

[[('HAAR', 'ADJ'), ('ELINOR', 'PROPN')],
 [('ELINOR', 'PROPN'), ('GLYN', 'PROPN')],
 [('GLYN', 'PROPN'), ('EVANGELINES', 'X')],
 [('EVANGELINES', 'X'), ('GENVORDIGHEDER', 'NOUN')],
 [('GENVORDIGHEDER', 'NOUN'), ('AUTORISERET', 'PROPN')],
 [('AUTORISERET', 'PROPN'), ('OVERSÆTTELSE', 'NOUN')],
 [('OVERSÆTTELSE', 'NOUN'), ('FOR', 'ADP')],
 [('FOR', 'ADP'), ('NORGE', 'PROPN')],
 [('NORGE', 'PROPN'), ('OG', 'CCONJ')],
 [('OG', 'CCONJ'), ('DANMARK', 'PROPN')],
 [('DANMARK', 'PROPN'), ('AF', 'ADP')],
 [('AF', 'ADP'), ('HEDVIG', 'PROPN')],
 [('HEDVIG', 'PROPN'), ('MAGNUSSEN', 'NOUN')],
 [('MAGNUSSEN', 'NOUN'), ('NY', 'NOUN')],
 [('NY', 'NOUN'), ('UDGAVE', 'ADJ')]]

Now that we have our list of bigrams, we're going to make a function `get_neighbor_words()`. This function will return the most frequent words that appear next to a particular keyword. The function can also be fine-tuned to return neighbor words that match a certain part of speech by changing the `pos_label` parameter.

In [22]:
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        #Extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]        
        
        #Check to see if keyword is in the bigram
        if keyword in words:
            
            for word, label in bigram:
                
                #Now focus on the neighbor word, not the keyword
                if word.lower() != keyword:
                    #If the neighbor word matches the right pos_label, append it to the master list
                    if label == pos_label or pos_label == None:
                        neighbor_words.append(word.lower())
    
    return Counter(neighbor_words).most_common()

In [23]:
get_neighbor_words("godt", bigrams)

[('saa', 12),
 ('kan', 12),
 ('lide', 12),
 ('meget', 10),
 ('jeg', 9),
 ('ud', 8),
 ('det', 7),
 ('og', 6),
 ('om', 6),
 ('paa', 5),
 ('nok', 5),
 ('videre', 4),
 ('véd', 3),
 ('er', 3),
 ('af', 3),
 ('humør', 2),
 ('synes', 2),
 ('sige', 2),
 ('et', 2),
 ('mig', 2),
 ('at', 2),
 ('syntes', 2),
 ('selskab', 2),
 ('ikke', 2),
 ('forstaa', 2),
 ('altsammen', 2),
 ('men', 2),
 ('igen', 2),
 ('ganske', 2),
 ('se', 1),
 ('i', 1),
 ('alt', 1),
 ('forfærdelig', 1),
 ('hjem', 1),
 ('nu', 1),
 ('hvis', 1),
 ('overordentlig', 1),
 ('rigtig', 1),
 ('til', 1),
 ('skrevet', 1),
 ('de', 1),
 ('straks', 1),
 ('ja', 1),
 ('mønster', 1),
 ('kunde', 1),
 ('være', 1),
 ('lød', 1),
 ('gøre', 1),
 ('gaa', 1),
 ('passet', 1),
 ('kunnet', 1),
 ('have', 1),
 ('hvad', 1),
 ('sted', 1),
 ('sagde', 1),
 ('sovet', 1),
 ('helt', 1),
 ('var', 1),
 ('passe', 1),
 ('sir', 1),
 ('ser', 1),
 ('da', 1),
 ('som', 1),
 ('næsten', 1),
 ('skrive', 1),
 ('maa', 1),
 ('spurgte', 1),
 ('pas', 1),
 ('han', 1),
 ('forholdsvis',

In [24]:
get_neighbor_words("godt", bigrams, pos_label='VERB')

[('lide', 12),
 ('véd', 3),
 ('synes', 2),
 ('sige', 2),
 ('syntes', 2),
 ('se', 1),
 ('skrevet', 1),
 ('være', 1),
 ('lød', 1),
 ('gøre', 1),
 ('gaa', 1),
 ('passet', 1),
 ('have', 1),
 ('sagde', 1),
 ('sovet', 1),
 ('er', 1),
 ('passe', 1),
 ('ser', 1),
 ('skrive', 1),
 ('spurgte', 1),
 ('pas', 1)]

## Your Turn!

Try out `find_sentences_with_keyword()` and `get_neighbor_words` with your own keywords of interest.

In [25]:
find_sentences_with_keyword(keyword="jeg", document=document)

**jeg** vilde saa gerne vide, om det er morsomt at være en Eventyrerske, for det er øjensynligt det, **jeg** nu maa blive.

**jeg** har læst alt muligt om det i en Bog; det er at se godt ud, og ikke at have noget at leve af, og dog have Fornøjelse af Livet -- og det har **jeg** i Sinde!

**jeg** har ganske vist ingenting at leve af, for man kan ikke regne 300 Pund om Aaret for noget videre -- **jeg** er overordentlig køn, og **jeg** véd det godt, og **jeg** forstaar at sætte mit Haar og tage mine Hatte paa og den Slags Ting, saa **jeg** er naturligvis Eventyrerske!

**jeg** var ikke bestemt til at spille den Rolle.

Saa var hun saa inkonsekvent, at hun ikke skrev et ordentligt Testamente -- derfor er det, at det Menneske skal have alt, og **jeg** ingenting.  

...

Robert overvælder mig med Gaver, og Hertugen vil lade mig bære alle Torquilstone Juvelerne, naar **jeg** er blevet gift, foruden de Smaragder, han selv har givet mig.

**jeg** holder virkelig meget af ham.  

Christopher sendte mig dette karakteristiske Brev sammen med de Ørenringe, der er hans Gave -- meget store Smaragder indfattet med Diamanter:  "Det gør mig saa ondt, at **jeg** ikke skal se Dem paa den lykkelige Dag, men **jeg** har været heldig nok til at opdage, at Paris endnu har Glæder for mig.  

Bær dem, de vil passe til Deres Øjne!                                                               C. C."  Og i Morgen er det min Bryllupsdag, og **jeg** tager bort paa Bryllupsrejse med Robert -- bort, ind i den syvende Himmel.

Og **jeg** er _sikker_ paa, at ingen af os vil gabe!

In [26]:
get_neighbor_words("jeg", bigrams, pos_label=None)

[('at', 307),
 ('og', 268),
 ('sagde', 193),
 ('er', 173),
 ('har', 156),
 ('vil', 139),
 ('ikke', 134),
 ('var', 115),
 ('men', 114),
 ('kan', 111),
 ('vilde', 104),
 ('da', 90),
 ('kunde', 80),
 ('havde', 78),
 ('véd', 59),
 ('det', 54),
 ('mig', 54),
 ('tror', 52),
 ('skal', 46),
 ('som', 46),
 ('saa', 45),
 ('hvis', 45),
 ('saá', 42),
 ('følte', 42),
 ('skulde', 41),
 ('maa', 36),
 ('hvad', 35),
 ('han', 34),
 ('kom', 33),
 ('om', 31),
 ('naar', 31),
 ('antager', 28),
 ('spurgte', 28),
 ('hvor', 27),
 ('gik', 26),
 ('dem', 26),
 ('blev', 25),
 ('synes', 24),
 ('svarede', 22),
 ('aa', 21),
 ('haaber', 19),
 ('vidste', 18),
 ('nu', 17),
 ('paa', 17),
 ('for', 17),
 ('fordi', 17),
 ('føler', 17),
 ('elsker', 16),
 ('hun', 16),
 ('maatte', 15),
 ('før', 14),
 ('til', 14),
 ('troede', 14),
 ('medens', 14),
 ('syntes', 14),
 ('kommer', 14),
 ('holder', 14),
 ('i', 13),
 ('ud', 13),
 ('gerne', 13),
 ('ham', 13),
 ('sad', 13),
 ('gad', 12),
 ('fik', 12),
 ('tog', 12),
 ('siger', 11),
 ('se